In [1]:
from skilift import GTFS, TransitGraph

In [2]:
import pandas as pd
from zipfile import ZipFile
from collections import defaultdict
from typing import Dict, Set
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
fn = "/home/bmander/skilift_data/transit/redding.zip"

graph = TransitGraph.load(fn)

In [4]:
graph.feed.day_start, graph.feed.day_end

(0, 70200)

In [20]:
# get epoch time on Aptil 19, 2023 at 1:00pm pacific time
t0 = pd.Timestamp("2023-04-19 13:00:00", tz="America/Los_Angeles")
t0

Timestamp('2023-04-19 13:00:00-0700', tz='America/Los_Angeles')

In [21]:
stop_id = graph.feed.stops_with_name("Downtown Transit Center").iloc[0].stop_id
stop_id

2000

In [22]:
pattern_ids = graph.feed.stop_pattern_ids[stop_id]
pattern_ids

{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 23, 24, 25, 26, 27, 28}

In [23]:
service_ids = graph.feed.get_service_ids(t0.date())
service_ids

{'c_1658_b_18260_d_31',
 'c_1658_b_18260_d_63',
 'c_868_b_79977_d_30',
 'c_868_b_79977_d_31'}

In [27]:
events = graph.feed.find_stop_events(stop_id, t0)
events

[TimetableEvent(pattern_id=5, service_id='c_1658_b_18260_d_31', row=1, col=0, time=Timestamp('2023-04-19 14:25:00-0700', tz='America/Los_Angeles')),
 TimetableEvent(pattern_id=8, service_id='c_1658_b_18260_d_31', row=7, col=0, time=Timestamp('2023-04-19 13:20:00-0700', tz='America/Los_Angeles')),
 TimetableEvent(pattern_id=9, service_id='c_1658_b_18260_d_31', row=7, col=0, time=Timestamp('2023-04-19 13:20:00-0700', tz='America/Los_Angeles')),
 TimetableEvent(pattern_id=10, service_id='c_1658_b_18260_d_31', row=7, col=16, time=Timestamp('2023-04-19 13:50:00-0700', tz='America/Los_Angeles')),
 TimetableEvent(pattern_id=11, service_id='c_1658_b_18260_d_31', row=3, col=0, time=Timestamp('2023-04-19 13:50:00-0700', tz='America/Los_Angeles')),
 TimetableEvent(pattern_id=12, service_id='c_1658_b_18260_d_31', row=4, col=0, time=Timestamp('2023-04-19 14:50:00-0700', tz='America/Los_Angeles')),
 TimetableEvent(pattern_id=13, service_id='c_1658_b_18260_d_31', row=7, col=0, time=Timestamp('2023-04

In [28]:
node = ("at_stop", stop_id, t0)
edges = graph.adjacent_forward(node)